In [1]:
#Import dependencies
import pandas as pd
import numpy as np

#only projecting columns we need
proj_columns = ['Country', 'Ladder Score','Freedom', 'GDP', 'Life Expectancy', 'Generosity', 'Year']

#reading 2021 by itself because it has regional indictor which other csvs are muissing
df_2021 = pd.read_csv(f"Resources/years/2021.csv")[proj_columns + ['Regional indicator']]

happiness_combined_df = df_2021

#projecting only two columns 
country_df = df_2021[['Country', 'Regional indicator']]

#creating dictionary to  match regional indicators to countries
country_dict = country_df.set_index('Country').to_dict()['Regional indicator']

#looping through the years exct 2021 which we have already done and adding
#to a new combined dataframe 
for yr in range(2017, 2021):
    df_yr = pd.read_csv(f"Resources/years/{yr}.csv")[proj_columns]
    #adding regional indicator value to all years from country dict bove
    df_yr['Regional indicator'] = df_yr['Country'].apply(lambda x: country_dict.get(x, ''))
    #combining
    happiness_combined_df = pd.concat([happiness_combined_df,
        df_yr]) 

#happiness_combined_df.to_csv('Resources/combined_happiness.csv')
happiness_combined_df.head(5)

,Country,Ladder Score,Freedom,GDP,Life Expectancy,Generosity,Year,Regional indicator
0,Finland,7.842,0.949,10.775,72.0,-0.098,2021,Western Europe
1,Denmark,7.620,0.946,10.933,72.7,0.030,2021,Western Europe
2,Switzerland,7.571,0.919,11.117,74.4,0.025,2021,Western Europe
3,Iceland,7.554,0.955,10.878,73.0,0.160,2021,Western Europe
4,Netherlands,7.464,0.913,10.932,72.4,0.175,2021,Western Europe


In [2]:
# #DATA CLEANING
# #identify the columns with no regional indicator
# #happiness_combined_df['Regional indicator'] == ''][['Country','Year']]
# #pd.pivot_table(happiness_combined_df, values='Country', columns='Year', aggfunc='count')
# before_clean_df = happiness_combined_df
# before_clean_df.pivot(index='Country', columns='Year', values='Country').to_csv('Resources/pivoted-for-missing-val.csv')


In [3]:
#looking at the pivoted data set, it was revealed the countries were named diffrently in diffrent years,
#to solve that, these countries were renamed to one consistent value across years.
# ALso, some countries did not have any data for some years. For instance, Belize, Angola, Congo, Qatar are missing
#for most recent years. From our google research, It may be due to political situation and or high crime rate in Belize, 
#Angola and Congo and Central African Republic.
#Renaming the countries
rename_cntry_dict = {
    'Hong Kong S.A.R. of China': 'Hong Kong',
    'Hong Kong S.A.R., China': 'Hong Kong',
    'Trinidad & Tobago': 'Trinidad and Tobago',
    'North Macedonia': 'Macedonia',
    'Northern Cyprus': 'North Cyprus',
    'Taiwan Province of China': 'Taiwan',
}
rename_ri_dict = {
    'Hong Kong': 'East Asia',
    'Trinidad and Tobago': 'Latin America and Caribbean',
    'Macedonia': 'Central and Eastern Europe',
    'North Cyprus': 'Western Europe',
    'Taiwan': 'East Asia',
    'Central African Republic': 'Sub-Saharan Africa',
    'South Sudan': 'Sub-Saharan Africa',
    'Congo (Kinshasa)': 'Sub-Saharan Africa'
}

# in order to loop through data frame, call iterrows function
# which provides us an index of the row and all data in the row
for i, row in happiness_combined_df.iterrows():
    # grab either the rename value of the country, or if it is not
    # in the dictionary, just use the value already in the row as default
    #of the .get() function. We have to store this in a variable to use it 
    # for regional indicator below
    country = rename_cntry_dict.get(row['Country'], row['Country'])
    # using dataframe .at[], index from iterrows, and column name to set value
    happiness_combined_df.at[i, 'Country'] = country
    happiness_combined_df.at[i, 'Regional indicator'] = rename_ri_dict.get(country, row['Regional indicator'])

happiness_combined_df


,Country,Ladder Score,Freedom,GDP,Life Expectancy,Generosity,Year,Regional indicator
0,Finland,7.8420,0.949000,10.775000,72.000000,-0.098000,2021,Western Europe
1,Denmark,7.6200,0.946000,10.933000,72.700000,0.030000,2021,Western Europe
2,Switzerland,7.5710,0.919000,11.117000,74.400000,0.025000,2021,Western Europe
3,Iceland,7.5540,0.955000,10.878000,73.000000,0.160000,2021,Western Europe
4,Norway,7.4640,0.913000,10.932000,72.400000,0.175000,2021,Western Europe
...,...,...,...,...,...,...,...,...
148,Central African Republic,3.4759,0.640881,6.625160,45.200001,0.082410,2020,Sub-Saharan Africa
149,Rwanda,3.3123,0.900589,7.600104,61.098846,0.055484,2020,Sub-Saharan Africa
150,Zimbabwe,3.2992,0.711458,7.865712,55.617260,-0.072064,2020,Sub-Saharan Africa
151,South Sudan,2.8166,0.451314,7.425360,51.000000,0.016519,2020,Sub-Saharan Africa


In [4]:
# save csv after cleaning
happiness_combined_df.to_csv('Resources/combined_happiness_after_clean.csv')


In [5]:
#Data_check: identify the columns with no regional indicator
happiness_combined_df[happiness_combined_df['Regional indicator'] == '']['Country'].unique()

array([], dtype=object)

In [6]:
#identify top 5 happy and bottom 5 countries by ladder score in each year.
happiness_combined_df['rank'] = happiness_combined_df.groupby(['Year'])['Ladder Score'].rank('max', ascending=False)#.sort_values(['Year', 'rank'])
happiness_combined_df = happiness_combined_df.set_index(['Year', 'rank']).sort_index()
happiness_combined_df.to_csv('Resources/combined_happiness_indexed.csv')
happiness_combined_df


Country  Ladder Score   Freedom       GDP  \
Year rank                                                                
2017 1.0                     Finland         7.537  0.635423  1.616463   
     2.0                     Denmark         7.522  0.626007  1.482383   
     3.0                 Switzerland         7.504  0.627163  1.480633   
     4.0                     Iceland         7.494  0.620071  1.564980   
     5.0                      Norway         7.469  0.617951  1.443572   
...                              ...           ...       ...       ...   
2021 145.0                    Malawi         3.512  0.715000  7.926000   
     146.0                     Yemen         3.467  0.824000  9.782000   
     147.0                  Botswana         3.415  0.897000  7.676000   
     148.0                  Tanzania         3.145  0.677000  7.943000   
     149.0  Central African Republic         2.523  0.382000  7.695000   

            Life Expectancy  Generosity            Regional indicator  
Year rank                                                              
2017 1.0           0.796667    0.362012                Western Europe  
     2.0           0.792566    0.355280                Western Europe  
     3.0           0.833552    0.475540                Western Europe  
     4.0           0.858131    0.290549                Western Europe  
     5.0           0.809158    0.245483                Western Europe  
...                     ...         ...                           ...  
2021 145.0        48.700000   -0.131000            Sub-Saharan Africa  
     146.0        59.269000   -0.246000  Middle East and North Africa  
     147.0        61.400000    0.061000            Sub-Saharan Africa  
     148.0        56.201000   -0.047000            Sub-Saharan Africa  
     149.0        52.493000   -0.102000            Sub-Saharan Africa  

[769 rows x 7 columns]